### ARCHIVE

**Get a list of all relevant weather stations**

In [356]:
# Keep ones that are in the U.S. 
df_stations = pandas.read_csv('/home/amy/thesis/temperature/gsod-stations.csv')
df_stations = df_stations.dropna()
df_stations = df_stations.loc[df_stations['CTRY'] == "US"]

# Keep only stations that have data from 2012 - 2018
df_stations['BEGIN']= pandas.to_datetime(df_stations['BEGIN'], format="%Y%m%d") 
df_stations['END']= pandas.to_datetime(df_stations['END'], format="%Y%m%d") 
df_stations = df_stations[(df_stations['BEGIN'] <= '2012-1-1') & (df_stations['END'] >= '2012-12-31')]
df_stations.columns=['USAF', 'WBAN', 'name','country','state', 'ICAO', 'lat', 'lon', 'elev', 'begin', 'end']

df_stations.head()

,USAF,WBAN,name,country,state,ICAO,lat,lon,elev,begin,end
14474,690150,93121,TWENTY NINE PALMS,US,CA,KNXP,34.300,-116.167,625.1,1990-01-02,2020-03-02
15082,700001,26492,PORTAGE GLACIER,US,AK,PATO,60.785,-148.839,31.4,2006-01-01,2020-03-02
15084,700197,26558,SELAWIK,US,AK,PASK,66.600,-159.986,7.6,2006-01-01,2020-03-02
15086,700260,27502,W POST-WILL ROGERS MEMORIAL AIRPORT,US,AK,PABR,71.283,-156.782,9.5,1945-01-01,2020-03-02
15090,700300,27503,WAINWRIGHT AIRPORT,US,AK,PAWI,70.639,-159.995,9.1,1999-11-02,2020-03-02


**Match weather stations with egrid plant data**

In [365]:
df_data = df_data[(df_data.mwh >= 10.0) & (df_data.heat_rate <= 30.0)]    

df_cems_rounded = df_data.round()
df_stations_rounded = df_stations.round()
df_stations_rounded = df_stations_rounded.drop_duplicates(subset=['lat', 'lon'])

merged = pandas.merge(df_cems_rounded, df_stations_rounded, on=['lat','lon'], how='left', indicator='Exist')
print(merged['Exist'].value_counts(normalize=True) * 100)

# Drop ones that are only in one df, then use the station 
merged = merged.dropna()

both          97.297
left_only      2.703
right_only     0.000
Name: Exist, dtype: float64


In [367]:
# Reformat df to correspond to file storage in ftp 
df_stations.WBAN = df_stations.WBAN.astype(int)
df_stations.WBAN = df_stations.WBAN.astype(str)
df_stations.WBAN = df_stations.WBAN.str.zfill(5)

df_stations['station_id'] = df_stations.USAF + '-' + df_stations.WBAN

df_stations.head()

,USAF,WBAN,name,country,state,ICAO,lat,lon,elev,begin,end,station_id
14474,690150,93121,TWENTY NINE PALMS,US,CA,KNXP,34.300,-116.167,625.1,1990-01-02,2020-03-02,690150-93121
15082,700001,26492,PORTAGE GLACIER,US,AK,PATO,60.785,-148.839,31.4,2006-01-01,2020-03-02,700001-26492
15084,700197,26558,SELAWIK,US,AK,PASK,66.600,-159.986,7.6,2006-01-01,2020-03-02,700197-26558
15086,700260,27502,W POST-WILL ROGERS MEMORIAL AIRPORT,US,AK,PABR,71.283,-156.782,9.5,1945-01-01,2020-03-02,700260-27502
15090,700300,27503,WAINWRIGHT AIRPORT,US,AK,PAWI,70.639,-159.995,9.1,1999-11-02,2020-03-02,700300-27503


**Try using Plant: 3935_1 (coal) and Weather station: 720160-63884**
- Checked the location manually, they are pretty close

In [368]:
### Function from https://github.com/tagomatech/ETL/blob/master/gsod/gsod.py 
def getData(station, year):
    '''
    Get weather data from the internet as memory stream
    '''
    big_df = pandas.DataFrame()


    # Define URL
    url = 'http://www1.ncdc.noaa.gov/pub/data/gsod/' + str(year) + '/' + str(station) \
        + '-' + str(year) + '.op.gz'

    # Define data stream
    stream = requests.get(url)

    # Unzip on-the-fly
    decomp_bytes = gzip.decompress(stream.content)
    data = decomp_bytes.decode('utf-8').split('\n')

    '''
    Data manipulations and ordering
    '''
    # Remove start and end
    data.pop(0) # Remove first line header
    data.pop()  # Remove last element

    # Define lists
    (stn, wban, date, temp, temp_c, dewp, dewp_c,
     slp, slp_c, stp, stp_c, visib, visib_c,
     wdsp, wdsp_c, mxspd, gust, max, max_f, min, min_f,
     prcp, prcp_f, sndp, f, r, s, h, th, tr) = ([] for i in range(30))

    # Fill in lists
    for i in range(0, len(data)):
        stn.append(data[i][0:6])
        wban.append(data[i][7:12])
        date.append(data[i][14:22])         
        temp.append(data[i][25:30])
        dewp.append(data[i][36:41])
        slp.append(data[i][46:52])      # Mean sea level pressure
        stp.append(data[i][57:63])      # Mean station pressure
        visib.append(data[i][68:73])
        wdsp.append(data[i][78:83])
        mxspd.append(data[i][88:93])
        gust.append(data[i][95:100])
        max.append(data[i][103:108])
        min.append(data[i][111:116])
        prcp.append(data[i][118:123])
        sndp.append(data[i][125:130])   # Snow depth in inches to tenth
        f.append(data[i][132])          # Fog
        r.append(data[i][133])          # Rain or drizzle
        s.append(data[i][134])          # Snow or ice pallet
        h.append(data[i][135])          # Hail
        th.append(data[i][136])         # Thunder
        tr.append(data[i][137])         # Tornado or funnel cloud

    '''
    Replacements
    min_f & max_f
    blank   : explicit => e
    *       : derived => d
    '''
    #Create dataframe & cleanse data
    # Create intermediate matrix
    mat = np.matrix(data=[stn, wban, date, temp, dewp,
           slp, stp, visib,
           wdsp, mxspd, gust, max, min,
           prcp, sndp, f, r, s, h, th, tr]).T

    # Define header names
    headers = ['stn', 'wban', 'date', 'temp', 'dewp', 
    'slp', 'stp',  'visib', 
    'wdsp', 'mxspd', 'gust', 'max', 'min', 
    'prcp', 'sndp', 'f', 'r', 's', 'h', 'th', 'tr']

    # Set precision
    pandas.set_option('precision', 3)

    # Create dataframe from matrix object
    df = pandas.DataFrame(data=mat, columns=headers)

    # Replace missing values with NAs
    df = df.where(df != ' ', 9999.9)

    # Create station ids
    df['station_id'] = df['stn'].map(str) + '-' + df['wban'].map(str)
    df = df.drop(['stn', 'wban'], axis=1)

    # Convert to numeric
    df[['temp', 'dewp', 'slp', 'stp', 'visib', 'wdsp','mxspd',  'gust', 'max', 'min', 'prcp', 'sndp']] = df[['temp', 'dewp', 'slp', 'stp', 'visib', 'wdsp', 'mxspd', 'gust', 'max','min', 'prcp', 'sndp']].apply(pandas.to_numeric)
    
    # Replace missing weather data with NaNs
    df = df.replace(to_replace=[99.99, 99.9,999.9,9999.9], value=np.nan)

    # Convert to date format
    df['date'] = pandas.to_datetime(df['date'], format='%Y%m%d')

    big_df = pandas.concat([big_df, df])

    return big_df

In [369]:
weather_data = pandas.DataFrame()

year = 2012
try:
    weather_data = weather_data.append(getData(station=str('720160-63884'), year=str(year)))
except:
    print('no data')
    pass
weather_data

,date,temp,dewp,slp,stp,visib,wdsp,mxspd,gust,max,min,prcp,sndp,f,r,s,h,th,tr,station_id
0,2012-01-01,39.3,37.3,NaN,NaN,9.3,0.9,9.9,21.0,57.2,32.0,NaN,NaN,0,1,0,0,0,0,720160-63884
1,2012-01-02,34.5,22.3,NaN,NaN,9.2,4.8,9.9,22.9,44.6,26.6,NaN,NaN,0,0,1,0,0,0,720160-63884
2,2012-01-03,22.5,17.1,NaN,NaN,5.6,3.2,8.0,15.9,24.8,21.2,NaN,NaN,0,0,1,0,0,0,720160-63884
3,2012-01-04,23.3,15.3,NaN,NaN,10.0,0.2,4.1,NaN,42.8,12.2,0.0,NaN,0,0,0,0,0,0,720160-63884
4,2012-01-05,34.3,28.4,NaN,NaN,10.0,0.3,4.1,NaN,46.4,26.6,0.0,NaN,0,0,0,0,0,0,720160-63884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,2012-12-27,33.0,30.6,NaN,NaN,7.5,NaN,NaN,NaN,46.4,30.2,NaN,NaN,0,1,1,0,0,0,720160-63884
345,2012-12-28,32.7,28.7,NaN,NaN,10.0,NaN,NaN,NaN,41.0,30.2,0.0,NaN,0,0,0,0,0,0,720160-63884
346,2012-12-29,36.5,35.2,NaN,NaN,5.8,NaN,NaN,NaN,39.2,32.0,NaN,NaN,0,1,1,0,0,0,720160-63884
347,2012-12-30,28.9,25.7,NaN,NaN,5.8,NaN,NaN,NaN,32.0,23.0,NaN,NaN,0,0,1,0,0,0,720160-63884


In [370]:
plant = df_data[df_data['orispl_unit']=='3935_1'] 
cols = ['date', 'hour', 'mwh', 'heat_rate', 'co2', 'so2', 'nox', 'orispl_unit']
plant = plant[cols]
plant['date'] = pandas.to_datetime(plant['date'])

# separate the dfs
demand = plant[['date', 'mwh']]
plant_info = plant[['date', 'mwh', 'heat_rate', 'co2', 'so2', 'nox']]
demand['date'] = pandas.to_datetime(demand['date'])
plant_info['date'] = pandas.to_datetime(plant_info['date'])

# Aggregate by day: sum the demand, average the heat and emissions rates 
demand = demand.groupby('date').sum().reset_index()
plant_info = plant_info.groupby('date').mean().reset_index()

df = pandas.merge(weather_data, demand, on='date', how='left')
df = pandas.merge(weather_data, plant_info, on='date', how='inner')
df

/home/amy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/amy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,date,temp,dewp,slp,stp,visib,wdsp,mxspd,gust,max,...,s,h,th,tr,station_id,mwh,heat_rate,co2,so2,nox
0,2012-01-01,39.3,37.3,NaN,NaN,9.3,0.9,9.9,21.0,57.2,...,0,0,0,0,720160-63884,510.083,9.060,861.985,0.020,0.194
1,2012-01-02,34.5,22.3,NaN,NaN,9.2,4.8,9.9,22.9,44.6,...,1,0,0,0,720160-63884,631.125,8.774,834.792,0.088,0.184
2,2012-01-03,22.5,17.1,NaN,NaN,5.6,3.2,8.0,15.9,24.8,...,1,0,0,0,720160-63884,741.167,8.584,816.715,0.157,0.166
3,2012-01-04,23.3,15.3,NaN,NaN,10.0,0.2,4.1,NaN,42.8,...,0,0,0,0,720160-63884,722.167,8.582,816.562,0.139,0.172
4,2012-01-05,34.3,28.4,NaN,NaN,10.0,0.3,4.1,NaN,46.4,...,0,0,0,0,720160-63884,668.833,8.699,827.682,0.126,0.180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2012-12-27,33.0,30.6,NaN,NaN,7.5,NaN,NaN,NaN,46.4,...,1,0,0,0,720160-63884,621.083,9.104,866.220,0.182,0.168
269,2012-12-28,32.7,28.7,NaN,NaN,10.0,NaN,NaN,NaN,41.0,...,0,0,0,0,720160-63884,619.708,9.153,870.893,0.236,0.173
270,2012-12-29,36.5,35.2,NaN,NaN,5.8,NaN,NaN,NaN,39.2,...,1,0,0,0,720160-63884,635.417,9.099,865.743,0.269,0.183
271,2012-12-30,28.9,25.7,NaN,NaN,5.8,NaN,NaN,NaN,32.0,...,1,0,0,0,720160-63884,629.958,9.194,874.745,0.229,0.180


In [371]:
# Remove columns with a lot of NaN
df = df.drop(columns = ['slp', 'stp', 'prcp', 'sndp', 'gust', 'th', 'tr', 'h', 'f'])

# Replace Nan with mean 
df = df.fillna(df.mean())

# Extract month, hour, and day from date columns
df['month'] = pandas.DatetimeIndex(df.date).month 
df['day'] = pandas.DatetimeIndex(df.date).day
df['year'] = pandas.DatetimeIndex(df.date).year

df

,date,temp,dewp,visib,wdsp,mxspd,max,min,r,s,station_id,mwh,heat_rate,co2,so2,nox,month,day,year
0,2012-01-01,39.3,37.3,9.3,0.900,9.900,57.2,32.0,1,0,720160-63884,510.083,9.060,861.985,0.020,0.194,1,1,2012
1,2012-01-02,34.5,22.3,9.2,4.800,9.900,44.6,26.6,0,1,720160-63884,631.125,8.774,834.792,0.088,0.184,1,2,2012
2,2012-01-03,22.5,17.1,5.6,3.200,8.000,24.8,21.2,0,1,720160-63884,741.167,8.584,816.715,0.157,0.166,1,3,2012
3,2012-01-04,23.3,15.3,10.0,0.200,4.100,42.8,12.2,0,0,720160-63884,722.167,8.582,816.562,0.139,0.172,1,4,2012
4,2012-01-05,34.3,28.4,10.0,0.300,4.100,46.4,26.6,0,0,720160-63884,668.833,8.699,827.682,0.126,0.180,1,5,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,2012-12-27,33.0,30.6,7.5,1.072,6.085,46.4,30.2,1,1,720160-63884,621.083,9.104,866.220,0.182,0.168,12,27,2012
269,2012-12-28,32.7,28.7,10.0,1.072,6.085,41.0,30.2,0,0,720160-63884,619.708,9.153,870.893,0.236,0.173,12,28,2012
270,2012-12-29,36.5,35.2,5.8,1.072,6.085,39.2,32.0,1,1,720160-63884,635.417,9.099,865.743,0.269,0.183,12,29,2012
271,2012-12-30,28.9,25.7,5.8,1.072,6.085,32.0,23.0,0,1,720160-63884,629.958,9.194,874.745,0.229,0.180,12,30,2012


### Read in data, save as csv

In [14]:
# # Get plant data for 2014
egrid_fname = '/home/amy/simple_dispatch/egrid2014_data.xlsx' 
df_plnt = pandas.read_excel(egrid_fname, 'PLNT14', skiprows=[0])

#geography
df_plnt = df_plnt[['ORISPL', 'BACODE', 'ISORTO', 'SUBRGN', 'LAT', 'LON', 'PLFUELCT']]
df_plnt.columns = ['orispl', 'ba', 'isorto', 'egrid', 'lat', 'lon', 'fuel']

df_plnt = df_plnt.loc[df_plnt['isorto'] == 'PJM']
df_plnt.to_csv('df_plnt.csv')

In [15]:
# Get data for 4 years for North Carolina 
df_data = pandas.DataFrame()
year = 2012

cems_folder = '/home/amy/MEFR/Marginal emissions factor data/CEMS/unzipped/' + str(year) + '/'
s = ['de', 'oh', 'in', 'wv']
df_cems = pandas.DataFrame()

for state in s: 
    for m in ['01','02','03','04','05','06','07','08','09','10','11', '12']:
        df_cems_add = pandas.read_csv(cems_folder + str(year) + state + m + '.csv')
        df_cems_add = df_cems_add[['ORISPL_CODE', 'UNITID', 'OP_DATE','OP_HOUR','GLOAD (MW)', 'SO2_MASS (lbs)', 'NOX_MASS (lbs)', 'CO2_MASS (tons)', 'HEAT_INPUT (mmBtu)']].dropna()
        df_cems_add.columns=['orispl', 'unit', 'date','hour','mwh', 'so2_tot', 'nox_tot', 'co2_tot', 'mmbtu']
        df_cems = pandas.concat([df_cems, df_cems_add])

#create the 'orispl_unit' column, which combines orispl and unit into a unique tag for each generation unit
df_cems['orispl_unit'] = df_cems['orispl'].map(str) + '_' + df_cems['unit'].map(str)

df_cems = df_cems.merge(df_plnt, how='left', on='orispl')  
df_cems = df_cems[df_cems['isorto']=='PJM'] 

#convert emissions to kg
df_cems.co2_tot = df_cems.co2_tot * 907.185 #tons to kg
df_cems.so2_tot = df_cems.so2_tot * 0.454 #lbs to kg
df_cems.nox_tot = df_cems.nox_tot * 0.454 #lbs to kg

#calculate the hourly heat and emissions rates. Later we will take the medians over each week to define the generators weekly heat and emissions rates.
df_cems['heat_rate'] = df_cems.mmbtu / df_cems.mwh
df_cems['co2'] = df_cems.co2_tot / df_cems.mwh
df_cems['so2'] = df_cems.so2_tot / df_cems.mwh
df_cems['nox'] = df_cems.nox_tot / df_cems.mwh
df_cems.replace([scipy.inf, -scipy.inf], scipy.nan, inplace=True) #don't want inf messing up median calculations

#drop any bogus data. For example, the smallest mmbtu we would expect to see is 25MW(smallest unit) * 0.4(smallest minimum output) * 6.0 (smallest heat rate) = 60 mmbtu. Any entries with less than 60 mmbtu fuel or less than 6.0 heat rate, let's get rid of that row of data.
df_cems = df_cems[(df_cems.heat_rate >= 6.0) & (df_cems.mmbtu >= 60)]
df_data = df_data.append(df_cems)

/home/amy/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (10,12,18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
from datetime import timedelta
year = 2012 
df_orispl_unit = df_data.copy(deep=True)
df_orispl_unit.date = df_orispl_unit.date.str.replace('/','-')
temp = pandas.DataFrame(df_orispl_unit.date.str.split('-').tolist(), columns=['month', 'day', 'year'], index=df_orispl_unit.index).astype(float)
df_orispl_unit['monthday'] = temp.year*10000 + temp.month*100 + temp.day
###
#loop through the weeks, slice the data, and find the average heat rates and emissions rates
#first, add a column 't' that says which week of the simulation we are in
df_orispl_unit['t'] = 52
for t in np.arange(52)+1:
    start = (datetime.strptime(str(year) + '-01-01', '%Y-%m-%d') + timedelta(days=7.05*(t-1)-1)).strftime('%Y-%m-%d') 
    end = (datetime.strptime(str(year) + '-01-01', '%Y-%m-%d') + timedelta(days=7.05*(t)-1)).strftime('%Y-%m-%d') 
    start_monthday = float(start[0:4])*10000 + float(start[5:7])*100 + float(start[8:])
    end_monthday = float(end[0:4])*10000 + float(end[5:7])*100 + float(end[8:])
    #slice the data for the days corresponding to the time series period, t
    df_orispl_unit.loc[(df_orispl_unit.monthday >= start_monthday) & (df_orispl_unit.monthday < end_monthday), 't'] = t 

In [17]:
#remove outlier emissions and heat rates. These happen at hours where a generator's output is very low (e.g. less than 10 MWh). To remove these, we will remove any datapoints where mwh < 10.0 and heat_rate < 30.0 (0.5% percentiles of the 2014 TRE data).
df_orispl_unit = df_orispl_unit[(df_orispl_unit.mwh >= 10.0) & (df_orispl_unit.heat_rate <= 30.0)]    

#aggregate by orispl_unit and t to get the heat rate, emissions rates, and capacity for each unit at each t
temp_2 = df_orispl_unit.groupby(['orispl_unit', 't'], as_index=False).agg('median')[['orispl_unit', 't', 'heat_rate', 'co2', 'so2', 'nox']].copy(deep=True)
temp_2['mw'] = df_orispl_unit.groupby(['orispl_unit', 't'], as_index=False).agg('max')['mwh'].copy(deep=True)
#condense df_orispl_unit down to where we just have 1 row for each unique orispl_unit
df_orispl_unit = df_orispl_unit.groupby('orispl_unit', as_index=False).agg('max')[['orispl_unit', 'orispl', 'ba', 'isorto', 'egrid', 'mwh']]
df_orispl_unit.rename(columns={'mwh':'mw'}, inplace=True)
for c in ['heat_rate', 'co2', 'so2', 'nox', 'mw']:
    temp_3 = temp_2.set_index(['orispl_unit', 't'])[c].unstack().reset_index()
    temp_3.columns = list(['orispl_unit']) + ([c + str(a) for a in scipy.arange(52)+1])
    
    #remove any outlier values in the 1st or 99th percentiles
    max_array = temp_3.copy().drop(columns='orispl_unit').quantile(0.99, axis=1) 
    min_array = temp_3.copy().drop(columns='orispl_unit').quantile(0.01, axis=1)
    
    median_array = temp_3.copy().drop(columns='orispl_unit').median(axis=1)
    for i in temp_3.index: 
        test = temp_3.drop(columns='orispl_unit').iloc[i]
        test[test > max_array[i]] = np.NaN
        test[test < min_array[i]] = np.NaN
        test = list(test) #had a hard time putting the results back into temp_3 without using a list
        #if the first entry in test is nan, we want to fill that with the median value so that we can use ffill later
        if math.isnan(test[0]):
            test[0] = median_array[i]
        test.insert(0, temp_3.iloc[i].orispl_unit)
        temp_3.iloc[i] = test
        
    #for any nan values (assuming these are offline generators without any output data), fill nans with a large heat_rate that will move the generator towards the end of the merit order and large-ish emissions rate, so if the generator is dispatched in the model it will jack up prices but emissions won't be heavily affected (note, previously I just replaced all nans with 99999, but I was concerned that this might lead to a few hours of the year with extremely high emissions numbers that threw off the data)
#     M = float(scipy.where(c=='heat_rate', 50.0, scipy.where(c=='co2', 1500.0, scipy.where(c=='so2', 4.0, scipy.where(c=='nox', 3.0, scipy.where(c=='mw', 0.0, 99.0)))))) #M here defines the heat rate and emissions data we will give to generators that were not online in the historical data
#     #if we are using hist_downtime, then replace scipy.NaN with M. That way offline generators can still be dispatched, but they will have high cost and high emissions.
#     if self.hist_downtime:
#         temp_3 = temp_3.fillna(M)

    #if we are not using hist_downtime, then use ffill to populate the scipy.NaN values. This allows us to use the last observed value for the generator to populate data that we don't have for it. For example, if generator G had a heat rate of 8.5 during time t-1, but we don't have data for time t, then we assume that generator G has a heat rate of 8.5 for t. When we do this, we can begin to include generators that might be available for dispatch but were not turned on because prices were too low. However, we also remove any chance of capturing legitimate maintenance downtime that would impact the historical data. So, for validation purposes, we probably want to have hist_downtime = True. For future scenario analysis, we probably want to have hist_downtime = False.
    temp_3 = temp_3.fillna(method='ffill')  
    temp_3.iloc[0] = temp_3.iloc[0].fillna(method='ffill') #for some reason the first row was not doing fillna(ffill)
    #merge temp_3 with df_orispl_unit. Now we have weekly heat rates, emissions rates, and capacities for each generator. These values depend on whether we are including hist_downtime
    df_orispl_unit = df_orispl_unit.merge(temp_3, on='orispl_unit', how='left')

/home/amy/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: scipy.arange is deprecated and will be removed in SciPy 2.0.0, use numpy.arange instead
  if sys.path[0] == '':


In [18]:
df_orispl_unit.to_csv('heat_rate.csv')

In [19]:
#merge df_orispl_unit into df. Now we have a dataframe with weekly heat rate and emissions rates for any plants in CEMS with that data. There will be some nan values in df for those weekly columns (e.g. 'heat_rate1', 'co223', etc. that we will want to fill with annual averages from eGrid for now
orispl_units_egrid = df.orispl_unit.unique()
orispl_units_cems = df_orispl_unit.orispl_unit.unique()
df_leftovers = df[df.orispl_unit.isin(scipy.setdiff1d(orispl_units_egrid, orispl_units_cems))]

NameError: name 'df' is not defined